# Tropical Cyclone U-Net Wind Speed Prediction (TCUWSP-Net) Model for Calculating the Intensity (MSWS) of Tropical Cyclones

# Building the TCUWSP-Net Model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def encoder_block(inputs, filters):
    x = layers.Conv3D(filters=filters, kernel_size=(3, 3, 4), padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    return x

def convlstm_block(inputs, filters):
    # Reshape to (timesteps, height, width, channels) for ConvLSTM
    x = layers.Reshape((inputs.shape[1], inputs.shape[2], inputs.shape[3], inputs.shape[4]))(inputs)
    x = layers.ConvLSTM2D(filters=filters, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x = layers.BatchNormalization()(x)
    # Reshape back to 3D conv format
    x = layers.Reshape((inputs.shape[1], inputs.shape[2], inputs.shape[3], filters))(x)
    return x

def decoder_block(inputs, skip_connection, filters):
    x = layers.Conv3DTranspose(filters=filters, kernel_size=(3, 3, 4), padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    skip_resized = layers.Conv3D(filters, (1, 1, 1), padding="same")(skip_connection)
    x = layers.Concatenate()([x, skip_resized]) 
    x = layers.ConvLSTM2D(filters=filters, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    return x

def build_unet_convlstm(input_shape=(8, 95, 95, 3)):
    input_tensor = layers.Input(shape=input_shape)

    # Encoder with ConvLSTM
    skip1 = encoder_block(input_tensor, filters=8)
    skip1 = convlstm_block(skip1, filters=8)  # Added ConvLSTM
    
    skip2 = encoder_block(skip1, filters=16)
    skip2 = convlstm_block(skip2, filters=16)  # Added ConvLSTM

    # Bottleneck with ConvLSTM
    x = layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding="same", activation="relu")(skip2)
    x = layers.BatchNormalization()(x)
    x = convlstm_block(x, filters=32)  # Bottleneck ConvLSTM

    # Decoder
    x = decoder_block(x, skip2, filters=16)
    x = decoder_block(x, skip1, filters=8)

    # Final Output Layer
    x = layers.Conv3D(filters=1, kernel_size=(1, 1, 1), activation="relu")(x)
    x = layers.GlobalAveragePooling3D()(x)

    model = models.Model(inputs=input_tensor, outputs=x)
    return model

# Compile & Summarize
input_shape = (8, 95, 95, 3)
model = build_unet_convlstm(input_shape=input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="mse")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8, 95, 95, 3)   │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 8, 95, 95, 8)   │            872 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 8, 95, 95, 8)   │             32 │ conv3d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 8, 95, 95, 8)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_lstm2d (ConvLSTM2D)  │ (None, 8, 95, 95, 8)   │          4,640 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 8, 95, 95, 8)   │             32 │ conv_lstm2d[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 8, 95, 95, 8)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 8, 95, 95, 16)  │          4,624 │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 8, 95, 95, 16)  │             64 │ conv3d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 8, 95, 95, 16)  │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_lstm2d_1             │ (None, 8, 95, 95, 16)  │         18,496 │ reshape_2[0][0]        │
│ (ConvLSTM2D)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 8, 95, 95, 16)  │             64 │ conv_lstm2d_1[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_3 (Reshape)       │ (None, 8, 95, 95, 16)  │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 8, 95, 95, 32)  │         13,856 │ reshape_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 8, 95, 95, 32)  │            128 │ conv3d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_4 (Reshape)       │ (None, 8, 95, 95, 32)  │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_lstm2d_2        

 Total params: 174,961 (683.44 KB)

 Trainable params: 174,689 (682.38 KB)

 Non-trainable params: 272 (1.06 KB)

# Building custom Radial Spatio-temporal Network (RSTNet)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def RSTNet(input_shape):
    """
    Creates the subnet for extracting TC radial structure features using a five-branch CNN design with 2D convolutions.

    Parameters:
    - input_shape: tuple, shape of the input data (e.g., (95, 95, 3))

    Returns:
    - model: tf.keras.Model, the radial structure subnet model
    """
    
    input_tensor = layers.Input(shape=input_shape)

    # Divide input data into four quadrants (NW, NE, SW, SE)
    # Assuming the input shape is (batch_size, height, width, channels)
    
    # Quadrant extraction - using slicing to separate quadrants
    nw_quadrant = input_tensor[:, :input_shape[0]//2, :input_shape[1]//2, :]
    ne_quadrant = input_tensor[:, :input_shape[0]//2, input_shape[1]//2:, :]
    sw_quadrant = input_tensor[:, input_shape[0]//2:, :input_shape[1]//2, :]
    se_quadrant = input_tensor[:, input_shape[0]//2:, input_shape[1]//2:, :]


    target_height = max(input_shape[0]//2, input_shape[0] - input_shape[0]//2)  # 48
    target_width = max(input_shape[1]//2, input_shape[1] - input_shape[1]//2)  # 48
    
    # Padding the quadrants to match the target size (48, 48)
    nw_quadrant = layers.ZeroPadding2D(padding=((0, target_height - nw_quadrant.shape[1]), 
                                                (0, target_width - nw_quadrant.shape[2])))(nw_quadrant)
    ne_quadrant = layers.ZeroPadding2D(padding=((0, target_height - ne_quadrant.shape[1]), 
                                                (0, target_width - ne_quadrant.shape[2])))(ne_quadrant)
    sw_quadrant = layers.ZeroPadding2D(padding=((0, target_height - sw_quadrant.shape[1]), 
                                                (0, target_width - sw_quadrant.shape[2])))(sw_quadrant)
    se_quadrant = layers.ZeroPadding2D(padding=((0, target_height - se_quadrant.shape[1]), 
                                                (0, target_width - se_quadrant.shape[2])))(se_quadrant)

    print(nw_quadrant.shape)
    print(ne_quadrant.shape)
    print(sw_quadrant.shape)
    print(se_quadrant.shape)

    # Main branch (processing the entire structure)
    main_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(input_tensor)
    y=layers.MaxPool2D()(main_branch)

    y = layers.ZeroPadding2D(padding=((0, target_height - y.shape[1]), 
                                   (0, target_width - y.shape[2])))(y)
    # Side branches (processing the individual quadrants)
    nw_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(nw_quadrant)
    ne_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(ne_quadrant)
    sw_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(sw_quadrant)
    se_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(se_quadrant)

    
    # Fusion operations (concatenate the outputs from the main branch and side branches)
    fusion = layers.concatenate([y, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)
    
    # Additional convolution layer to combine the fused features
    # x = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(fusion)
    x=layers.Reshape((1, 48, 48, 40))(fusion)
    x = layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x=layers.Reshape((48, 48, 16))(x)
    x=layers.MaxPool2D(pool_size=(2, 2))(x)

    # Final dense layer for further processing
    nw_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(nw_branch)
    
    ne_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(ne_branch)
    sw_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(sw_branch)
    se_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(se_branch)
    nw_branch = layers.MaxPool2D(pool_size=(2, 2))(nw_branch)
    ne_branch = layers.MaxPool2D(pool_size=(2, 2))(ne_branch)
    sw_branch = layers.MaxPool2D(pool_size=(2, 2))(sw_branch)
    se_branch = layers.MaxPool2D(pool_size=(2, 2))(se_branch)

    fusion = layers.concatenate([x, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)

    # x = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(fusion)
    x=layers.Reshape((1, 24, 24, 80))(fusion)
    x = layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x=layers.Reshape((24, 24, 32))(x)
    x=layers.MaxPool2D(pool_size=(2, 2))(x)
    
    nw_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(nw_branch)
    
    ne_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(ne_branch)
    sw_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(sw_branch)
    se_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(se_branch)
    nw_branch = layers.MaxPool2D(pool_size=(2, 2))(nw_branch)
    ne_branch = layers.MaxPool2D(pool_size=(2, 2))(ne_branch)
    sw_branch = layers.MaxPool2D(pool_size=(2, 2))(sw_branch)
    se_branch = layers.MaxPool2D(pool_size=(2, 2))(se_branch)

    fusion = layers.concatenate([x, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)

    # x = layers.Conv2D(filters=32, kernel_size=(3, 3),  activation='relu')(fusion)
    x=layers.Reshape((1,12, 12, 160))(fusion)
    x = layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x=layers.Reshape((12, 12, 32))(x)
    x=layers.Conv2D(filters=32, kernel_size=(3, 3), activation=None)(x)
    
    # Create and return the model
    x=layers.Flatten()(x)
    model = models.Model(inputs=input_tensor, outputs=x)
    return model

# Define input shape (batch_size, height, width, channels)
input_shape = (95, 95, 8)  # Example input shape (95x95 spatial resolution, 3 channels)

# Build the model
model = RSTNet(input_shape)

# Model summary
model.summary()


(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 95, 95, 8)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 95, 95, 8)      │            584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 47, 47, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_1 (GetItem)      │ (None, 47, 48, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_2 (GetItem)      │ (None, 48, 47, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_3 (GetItem)      │ (None, 48, 48, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 47, 47, 8)      │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 48, 48, 8)      │              0 │ get_item[0][0]         │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_1          │ (None, 48, 48, 8)      │              0 │ get_item_1[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_2          │ (None, 48, 48, 8)      │              0 │ get_item_2[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_3          │ (None, 48, 48, 8)      │              0 │ get_item_3[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_4          │ (None, 48, 48, 8)      │              0 │ max_pooling2d[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 48, 48, 8)      │            584 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 48, 48, 8)      │            584 │ zero_padding2d_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 48, 48, 8)      │            584 │ zero_padding2d_2[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 48, 48, 8)      │            584 │ zero_padding2d_3[0][0] │
├──────────────────────

 Total params: 418,184 (1.60 MB)

 Trainable params: 418,184 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

# Building the CNN Model

In [3]:
from tensorflow.keras import layers, models

def build_cnn_model(input_shape=(8, 8, 1)):
    # Define the input layer
    input_tensor = layers.Input(shape=input_shape)
    
    # Convolutional layer
    x = layers.Conv2D(64, (3, 3), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Flatten layer
    x = layers.Flatten()(x)
    
    # Create the model
    model = models.Model(inputs=input_tensor, outputs=x)
    
    return model

# cnn_model = build_cnn_model(input_shape=(8, 8, 1))
# cnn_model.summary()


# Building the Combined Model

In [4]:
from tensorflow.keras import layers, models, Input

def build_combined_model():
    # Define input shapes
    input_shape_3d = (8, 95, 95, 2)
    input_shape_radial = (95, 95, 8)
    input_shape_cnn = (8, 8, 1)
    
    input_shape_latitude = (8,)
    input_shape_longitude = (8,)
    input_shape_other = (9,)

    # Build individual models
    model_3d = build_unet_convlstm(input_shape=input_shape_3d)
    model_radial = RSTNet(input_shape=input_shape_radial)
    model_cnn = build_cnn_model(input_shape=input_shape_cnn)

    # Define new inputs
    input_latitude = Input(shape=input_shape_latitude ,name="latitude_input")
    input_longitude = Input(shape=input_shape_longitude, name="longitude_input")
    input_other = Input(shape=input_shape_other, name="other_input")

    # Flatten the additional inputs
    flat_latitude = layers.Dense(32,activation='relu')(input_latitude)
    flat_longitude = layers.Dense(32,activation='relu')(input_longitude)
    flat_other = layers.Dense(64,activation='relu')(input_other)

    # Combine all outputs
    combined = layers.concatenate([
        model_3d.output,
        model_radial.output,
        # model_cnn.output,
        # flat_latitude, 
        # flat_longitude, 
        flat_other
    ])

    # Add dense layers for final processing
    x = layers.Dense(128, activation='relu')(combined)  
    x = layers.Dense(1, activation=None)(x)

    # Create the final model
    final_model = models.Model(
        inputs=[model_3d.input,
                model_radial.input,
                # , model_cnn.input,
                # input_latitude, input_longitude,
                input_other 
               ],
        outputs=x
    )

    return final_model

# Build and summarize the updated model
# final_model = build_combined_model()
# final_model.summary()


# Reinitializing the Model

In [5]:
import tensorflow as tf
from keras import backend as K

# Clear session to remove any previously created computation graphs
K.clear_session()
tf.keras.backend.clear_session()

# Re-initialize your model
final_model = build_combined_model()


(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)


In [6]:
from tensorflow.keras import layers, models, optimizers

# Compiling the Model

In [7]:
final_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                    loss='mse',  # Use 'categorical_crossentropy' for multi-class
                    metrics=['mae'])



In [8]:
final_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8, 95, 95, 2)   │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 8, 95, 95, 8)   │            584 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 8, 95, 95, 8)   │             32 │ conv3d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 8, 95, 95, 8)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_lstm2d (ConvLSTM2D)  │ (None, 8, 95, 95, 8)   │          4,640 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 8, 95, 95, 8)   │             32 │ conv_lstm2d[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 8, 95, 95, 8)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 8, 95, 95, 16)  │          4,624 │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 8, 95, 95, 16)  │             64 │ conv3d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 8, 95, 95, 16)  │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 95, 95, 8)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_lstm2d_1             │ (None, 8, 95, 95, 16)  │         18,496 │ reshape_2[0][0]        │
│ (ConvLSTM2D)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 95, 95, 8)      │            584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 47, 47, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_1 (GetItem)      │ (None, 47, 48, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_2 (GetItem)      │ (None, 48, 47, 8)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_3 (GetItem)      │ (None, 48, 48, 8)      │              0 │ input_layer_1[0][0]    │
├──────────────────────

 Total params: 1,011,674 (3.86 MB)

 Trainable params: 1,011,402 (3.86 MB)

 Non-trainable params: 272 (1.06 KB)

# Loading our Extracted data from TCIR dataset¶

In [9]:
import numpy as np
reduced_images=np.load('/kaggle/input/project-data-set/reduced_images.npy')
hov_m_train=np.load('/kaggle/input/project-data-set/hov_m_train.npy')
train_vmax_3d=np.load('/kaggle/input/project-data-set/train_vmax_3d.npy')
lat_train=np.load('/kaggle/input/project-data-set/lat_train.npy')
lon_train=np.load('/kaggle/input/project-data-set/lon_train.npy')
int_diff_train=np.load('/kaggle/input/project-data-set/int_diff_train.npy')
y_train_avg=np.load('/kaggle/input/project-data-set/y_train_avg.npy')
reduced_images_test=np.load('/kaggle/input/project-data-set/reduced_images_test.npy')
hov_m_test=np.load('/kaggle/input/project-data-set/hov_m_test.npy')
test_vmax_3d=np.load('/kaggle/input/project-data-set/test_vmax_3d.npy')
lat_test=np.load('/kaggle/input/project-data-set/lat_test.npy')
lon_test=np.load('/kaggle/input/project-data-set/lon_test.npy')
int_diff_test=np.load('/kaggle/input/project-data-set/int_diff_test.npy')
y_test_avg=np.load('/kaggle/input/project-data-set/y_test_avg.npy')
reduced_images_valid=np.load('/kaggle/input/project-data-set/reduced_images_valid.npy')
hov_m_valid=np.load('/kaggle/input/project-data-set/hov_m_valid.npy')
valid_vmax_3d=np.load('/kaggle/input/project-data-set/valid_vmax_3d.npy')
lat_valid=np.load('/kaggle/input/project-data-set/lat_valid.npy')
lon_valid=np.load('/kaggle/input/project-data-set/lon_valid.npy')
int_diff_valid=np.load('/kaggle/input/project-data-set/int_diff_valid.npy')
y_valid_avg=np.load('/kaggle/input/project-data-set/y_valid_avg.npy')



In [10]:
ir=np.load("/kaggle/input/tropical-cyclone-intensity-regression/ir.npy")


# Integrating Gradient Maps to Satellite Imagery

In [11]:
import tensorflow as tf

def tf_gradient_magnitude(images):
    # Sobel kernels
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_y = tf.constant([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=tf.float32)
    sobel_x = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y = tf.reshape(sobel_y, [3, 3, 1, 1])

    images = tf.convert_to_tensor(images, dtype=tf.float32)
    images = tf.expand_dims(images, -1)

    gx = tf.nn.conv2d(images, sobel_x, strides=1, padding='SAME')
    gy = tf.nn.conv2d(images, sobel_y, strides=1, padding='SAME')
    grad_mag = tf.sqrt(tf.square(gx) + tf.square(gy) + 1e-6)

    return tf.squeeze(grad_mag, -1).numpy()

def GM_maps_prep(ir):
    GM_maps=[]
    for i in ir:
        GM_map = tf_gradient_magnitude(i)
        GM_maps.append(GM_map)
    GM_maps=np.array(GM_maps)
    return GM_maps
GM_maps = GM_maps_prep(ir)
print(GM_maps.shape)

(47381, 95, 95, 1)


In [12]:
train_GM_maps=GM_maps[:39814]
test_GM_maps=GM_maps[39814:]
# train_ir=ir[:39814]
# test_ir=ir[39814:]

In [13]:
# train_GM_maps.shape

In [14]:
# test_GM_maps.shape

In [15]:
import numpy as np

# Function to process and reshape image data
def process_images(images, batch_size=8, img_size=(95, 95, 1)):
    num_images = images.shape[0]
    
    # Trim the dataset to make it divisible by batch_size
    trimmed_size = (num_images // batch_size) * batch_size
    images_trimmed = images[:trimmed_size]

    # Reshape into (x, batch_size, img_size[0], img_size[1], img_size[2])
    images_reshaped = images_trimmed.reshape(-1, batch_size, *img_size)

    return images_reshaped[:-1]

# Assuming `train_images` and `test_images` are the original image datasets
train_images_processed = process_images(train_GM_maps)
test_images_processed = process_images(test_GM_maps)

print(f"Train Images Shape: {train_images_processed.shape}")
print(f"Test Images Shape: {test_images_processed.shape}")

Train Images Shape: (4975, 8, 95, 95, 1)
Test Images Shape: (944, 8, 95, 95, 1)


In [16]:
i_train=reduced_images[...,0:1]
pmw_train=reduced_images[...,1:2]

i_test=reduced_images_test[...,0:1]
pmw_test=reduced_images_test[...,1:2]

In [17]:
i_train=i_train+train_images_processed
i_test=i_test+test_images_processed

In [18]:
reduced_images = np.concatenate([i_train,pmw_train ], axis=-1)
reduced_images_test=np.concatenate([i_test,pmw_test], axis=-1)

In [19]:
import numpy as np

# Assuming `images` is the numpy array of shape (4580, 201, 201, 2)
print(np.isnan(reduced_images).sum())

4985066


In [20]:
reduced_images[np.isnan(reduced_images)] = 0  
reduced_images_test[np.isnan(reduced_images_test)] = 0

In [21]:
print()

# Fitting the Model and Prediction¶

In [22]:
final_model.fit(
    [reduced_images,hov_m_train,int_diff_train
     # , train_vmax_3d, lat_train, lon_train, int_diff_train
    ], 
    y_train_avg, 
    validation_data=(
        [reduced_images_valid,hov_m_valid,int_diff_valid
         # , valid_vmax_3d, lat_valid, lon_valid, int_diff_valid
        ], 
        y_valid_avg
    ),
    epochs=1, 
    batch_size=16,
)


311/311 ━━━━━━━━━━━━━━━━━━━━ 238s 629ms/step - loss: 289.9432 - mae: 11.5714 - val_loss: 95.3999 - val_mae: 7.0138


In [23]:
final_model.save("final_model.h5")


In [24]:
 y=final_model.predict([reduced_images_test,hov_m_test,int_diff_test
                        # ,test_vmax_3d,lat_test,lon_test,int_diff_test 
                       ])

30/30 ━━━━━━━━━━━━━━━━━━━━ 18s 358ms/step


# Model Evaluation

In [25]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [26]:
mae = mean_absolute_error(y_test_avg, y)

In [27]:
# Output the Mean Absolute Error
mae

7.202474911334151

In [28]:
rmse = mean_squared_error(y_test_avg, y, squared=False)

# Output the Root Mean Square Error
print(rmse)

9.936390207943665
